In [2]:
!nvidia-smi

Sun Apr 14 06:18:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
!pip install nvcc4jupyter

In [5]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpg0gi997z".


## **DFT**

In [75]:
%%cuda
#include<bits/stdc++.h>
#include<random>
#include<complex>
#include<cmath>
using namespace std ;

double Random(default_random_engine generator)
{
    // default_random_engine generator ;
    normal_distribution<double> distribution(0,1);
    double N =  distribution(generator) ;
    // distribution.reset();
    return N;
}

vector<complex<double>> DFT(vector<double> f)
{
    vector<complex<double>> X ;
    for(int k = 0 ; k < f.size();k++)
    {
        complex <double> temp  = complex(0.0,0.0);
        for(int n = 0 ; n < f.size();n++)
        {
            double angle = -1 * (2 * acos(0.0) * k * n)/ f.size() ;
            complex<double> Coef = complex(cos(angle),sin(angle));
            temp += f[n] * Coef ;
        }
        X.push_back(temp);;
    }
    return X ;
}

int main()
{
    int N = 8192;
    vector<double> Signal ;
    random_device rd{};
    for (int i =0;i<N;i++)
    {
        default_random_engine generator{rd()};
        Signal.push_back(Random(generator));
    }
    vector<complex<double>> X = DFT(Signal);
    for (int i =0;i<N;i++)
    {
        cout << X[i] <<"  ";
        if(i%10==0 && i!=0)
        {
            cout <<"\n";
        }
    }
    return 0;
}

(197.25,0)  (-85.67,-125.09)  (21.6473,53.9707)  (-59.0437,-108.821)  (-80.6263,128.089)  (106.12,9.88717)  (-2.11532,-50.8125)  (-13.9402,-46.3875)  (-65.616,-10.8552)  (-31.3127,8.30265)  (-54.8451,24.0551)  
(-17.9909,28.4081)  (-74.3363,57.4873)  (59.7904,147.399)  (131.014,-36.5589)  (-20.7544,-72.5444)  (-36.418,20.4619)  (15.9835,15.2928)  (-29.8788,9.05577)  (44.3036,86.5031)  (102.621,-56.533)  
(-21.8883,-65.2455)  (-18.0473,-37.5672)  (-66.173,3.52368)  (23.0248,44.1489)  (24.4382,-51.9091)  (-62.8083,-63.1197)  (-127.29,35.2741)  (-27.2756,126.548)  (42.0288,86.7329)  (99.7237,37.3458)  
(39.9705,-61.8748)  (-9.74186,-16.8408)  (-58.7852,-15.5896)  (3.73478,141.667)  (143.316,-52.1271)  (-79.5286,-27.1315)  (80.6335,36.3139)  (-60.1544,-56.5819)  (41.4373,118.609)  (73.554,-55.6854)  
(37.9285,28.0471)  (83.9507,-93.1577)  (-16.2823,-61.5064)  (-13.3149,-86.4413)  (-63.2031,-15.973)  (-33.7719,-60.6915)  (-124.726,50.5222)  (51.7483,62.4091)  (-27.995,-35.1527)  (-23.6423,4

## Parallel DFT

In [74]:
%%cuda
#include<bits/stdc++.h>
#include<random>
#include<complex>
#include<cuda_runtime.h>
#include<cmath>
#include<thrust/complex.h>
using namespace std ;

double Random(default_random_engine generator)
{
    // default_random_engine generator ;
    normal_distribution<double> distribution(0,1);
    double N =  distribution(generator) ;
    // distribution.reset();
    return N;
}

__global__ void DFT(double* f, thrust::complex<double> *X,int N)
{
    int k = (blockIdx.x * blockDim.x) + threadIdx.x ;
    if(k<N)
    {
        thrust::complex <double> temp  = thrust::complex<double>(0.0,0.0);
        for(int n = 0 ; n < N;n++)
        {
            double angle = -1 * (2 * acos(0.0) * k * n)/ N ;
            thrust::complex<double> Coef = thrust::complex<double>(cos(angle),sin(angle));
            temp.real(temp.real() + f[n]*Coef.real()) ;
            temp.imag(temp.imag() + f[n]*Coef.imag()) ;

        }
        X[k] = temp;
    }
}

int main()
{
    int N = 8192;
    double * Signal ;
    cudaMallocManaged(&Signal, N * sizeof(double));
    random_device rd{};
    for (int i =0;i<N;i++)
    {
        default_random_engine generator{rd()};
        Signal[i] = Random(generator);
    }
    thrust::complex<double> * X ;
    cudaMallocManaged(&X, N * sizeof(thrust::complex<double>));
    DFT<<<256,32>>>(Signal,X,N);
    cudaDeviceSynchronize();
    for (int i =0;i<N;i++)
    {
        cout << X[i] <<"  ";
        if(i%10==0 && i!=0)
        {
            cout <<"\n";
        }
    }
    return 0;
}

(12.664,0)  (129.206,-67.3586)  (-132.259,-91.1651)  (42.8311,82.4012)  (-15.6432,-80.6958)  (10.9603,17.9886)  (-51.0565,-87.3058)  (-57.2934,42.8469)  (-65.026,-11.4804)  (-37.3702,133.975)  (82.307,38.4268)  
(48.3839,16.7054)  (81.3491,-35.5914)  (15.3255,-124.318)  (-158.258,-34.9929)  (23.5164,126.914)  (11.9707,-83.1581)  (-51.6528,87.8597)  (79.302,-27.7577)  (-47.5035,-11.7796)  (1.46721,11.5482)  
(12.3829,45.5733)  (59.2753,-57.3305)  (-57.1089,-5.48389)  (41.4739,-7.24668)  (-43.1727,-23.6606)  (14.7997,3.2953)  (-42.0932,-8.32854)  (28.7534,36.3001)  (24.6605,-38.6998)  (-18.7727,-51.5317)  
(-73.7544,18.3951)  (49.2602,48.3092)  (13.8081,-58.6118)  (-5.74876,-8.09806)  (9.25333,-76.8658)  (-121.196,-67.7344)  (-100.259,98.6809)  (41.2272,46.1757)  (-25.5214,-3.97948)  (-16.2525,5.96195)  
(-83.021,52.2599)  (48.0169,117.061)  (46.7873,27.4108)  (115.478,51.7547)  (96.9329,-119.649)  (-24.8127,-85.8029)  (-57.1031,-76.2296)  (-97.2695,40.1656)  (44.6465,56.189)  (24.4805,-

## FFT

In [73]:
%%cuda
#include<bits/stdc++.h>
#include<random>
#include<complex>
#include<cmath>

using namespace std ;

double Random(default_random_engine generator)
{
    // default_random_engine generator ;
    normal_distribution<double> distribution(0,1);
    double N =  distribution(generator) ;
    // distribution.reset();
    return N;
}


complex<double> Cal(int theta,int N)
{
    double angle =  -1  * acos(0.0) * (theta/N);
    return complex<double>(cos(angle),sin(angle));
}



vector<complex<double>> Splitter(vector<complex<double>> f,int st)
{
    vector<complex<double>> X ;
    for (int i = st;i<f.size();i+=2)
    {
        X.push_back(f[i]);
    }
    return X;
}

vector<complex<double>> Split(vector<complex<double>> t)
{
    if(t.size()==1)
    {
        return t;
    }
    else
    {
        vector<complex<double>> even = Splitter(t,0);
        vector<complex<double>> odd = Splitter(t,1);
        even = Split(even);
        odd =  Split(odd);
        even.insert(even.end(),odd.begin(),odd.end());
        return even;
    }
}

complex<double> FFT(vector<complex<double>> f,int k,int N,int coef)
{
    if(f.size()==1)
    {
        return f[0];
    }
    else
    {
        vector<complex<double>> X ;
        for(int i=0;i<f.size();i+=2)
        {
            X.push_back(f[i] + Cal(coef*k,N) * f[i+1]);
        }
        return FFT(X,k,N,(int)coef/2);
    }
}

int main()
{
    int N = 8192;
    vector<complex<double>> Signal ;
    random_device rd{};
    for (int i =0;i<N;i++)
    {
        default_random_engine generator{rd()};
        Signal.push_back(complex<double>(Random(generator),0.0));
    }
    int pad = pow(2,ceil(log(N)/log(2))) - N ;
    for(int i = 0 ; i <pad;i++)
    {
        Signal.push_back(complex<double>(0.0,0.0));
    }
    vector<complex<double>> signal = Split(Signal);
    vector<complex<double>> X(Signal.size(),complex<double>(0.0,0.0));
    for (int k =0;k<N;k++)
    {
        X[k] = FFT(Signal,k,Signal.size()-pad,Signal.size());
    }
    for(int i = 0 ; i<N;i++)
    {
        cout << X[i] << " " ;
        if(i%10==0 && i!=0)
        {
            cout << "\n";
        }
    }
    return 0;
}

(-94.2146,0) (-145.231,-51.0166) (-102.048,94.2) (-51.0312,134.089) (13.8923,-58.0372) (-25.9963,-18.5062) (-80.6653,42.9876) (-36.4764,10.2842) (31.7911,27.6711) (68.7948,54.7993) (74.3178,7.70423) 
(34.2729,-18.91) (59.1596,-10.0091) (58.9326,-26.0976) (27.3384,-94.4736) (-28.7805,-74.5986) (59.9048,-43.3862) (-31.6449,-11.8968) (31.3366,14.1895) (28.7469,-38.3796) (39.0506,-45.8843) 
(38.8873,-63.7184) (-2.0334,35.0861) (112.367,14.6128) (-98.9259,-21.7163) (-2.07593,37.6919) (11.2558,-124.922) (-78.0039,-128.966) (-40.046,35.2817) (-23.5421,-16.5784) (-45.4746,-37.6067) 
(-60.9327,-8.07636) (1.12684,-20.3934) (-9.74034,20.6709) (10.8303,-32.8352) (-67.7275,-31.5332) (24.3949,5.50837) (-17.7472,-41.7646) (-30.4703,59.9801) (19.2315,47.1702) (-74.7841,2.65221) 
(-33.0465,79.2904) (6.14813,-60.7857) (-129.93,-96.7657) (-31.9832,236.061) (43.1553,93.0226) (-7.20025,113.25) (4.61824,146.48) (116.003,67.7316) (44.9158,-41.2025) (-56.5885,83.0375) 
(63.5358,128.639) (77.2077,58.4155) (22.

## Parallel FFT

In [ ]:
%%cuda
#include<bits/stdc++.h>
#include<random>
#include<complex>
#include<cmath>
#include<cuda_runtime.h>
#include<thrust/complex.h>
using namespace std ;

double Random(default_random_engine generator)
{
    // default_random_engine generator ;
    normal_distribution<double> distribution(0,1);
    double N =  distribution(generator) ;
    // distribution.reset();
    return N;
}


complex<double> Cal(int theta,int N)
{
    double angle =  -1  * acos(0.0) * (theta/N);
    return complex<double>(cos(angle),sin(angle));
}



vector<complex<double>> Splitter(vector<complex<double>> f,int st)
{
    vector<complex<double>> X ;
    for (int i = st;i<f.size();i+=2)
    {
        X.push_back(f[i]);
    }
    return X;
}

vector<complex<double>> Split(vector<complex<double>> t)
{
    if(t.size()==1)
    {
        return t;
    }
    else
    {
        vector<complex<double>> even = Splitter(t,0);
        vector<complex<double>> odd = Splitter(t,1);
        even = Split(even);
        odd =  Split(odd);
        even.insert(even.end(),odd.begin(),odd.end());
        return even;
    }
}

complex<double> FFT(vector<complex<double>> f,int k,int N,int coef)
{
    if(f.size()==1)
    {
        return f[0];
    }
    else
    {
        vector<complex<double>> X ;
        for(int i=0;i<f.size();i+=2)
        {
            X.push_back(f[i] + Cal(coef*k,N) * f[i+1]);
        }
        return FFT(X,k,N,(int)coef/2);
    }
}

int main()
{
    int N = 8192;
    vector<complex<double>> Signal ;
    random_device rd{};
    for (int i =0;i<N;i++)
    {
        default_random_engine generator{rd()};
        Signal.push_back(complex<double>(Random(generator),0.0));
    }
    int pad = pow(2,ceil(log(N)/log(2))) - N ;
    for(int i = 0 ; i <pad;i++)
    {
        Signal.push_back(complex<double>(0.0,0.0));
    }
    vector<complex<double>> signal = Split(Signal);
    vector<complex<double>> X(Signal.size(),complex<double>(0.0,0.0));
    for (int k =0;k<N;k++)
    {
        X[k] = FFT(Signal,k,Signal.size()-pad,Signal.size());
    }
    for(int i = 0 ; i<N;i++)
    {
        cout << X[i] << " " ;
        if(i%10==0 && i!=0)
        {
            cout << "\n";
        }
    }
    return 0;
}